In [1]:
import tensorflow as tf

In [2]:
class Conv(tf.keras.layers.Layer):
    def __init__(self,
                 filters:int,
                 kernel_size,
                 strides = (1,1),
                 padding = 'same'
                ):
        super(Conv, self).__init__()
        self.filters = filters
        self.kernel_size = kernel_size
        self.strides = strides
        self.padding = padding
        
        self.C = tf.keras.layers.Conv2D(filters = self.filters,
                                        kernel_size = self.kernel_size,
                                        strides = self.strides,
                                        padding = self.padding
                                       )
        self.BN = tf.keras.layers.BatchNormalization()
        
    def call(self, X):
        y = self.C(X)
        y = self.BN(y)
        y = tf.nn.relu(y)
        return y

In [3]:
class Stem(tf.keras.layers.Layer):
    def __init__(self):
        super(Stem, self).__init__()
        
        self.C1 = Conv(filters = 32,
                       kernel_size = (3,3),
                       strides = (2,2),
                       padding = 'valid'
                      )
        self.C2 = Conv(filters = 32,
                       kernel_size = (3,3),
                       padding = 'valid'
                      )
        self.C3 = Conv(filters = 64,
                       kernel_size = (3,3),
                      )
        self.MP4_1 = tf.keras.layers.MaxPool2D(pool_size = (3,3),
                                               strides = (2,2),
                                               padding = 'valid'
                                              )
        self.C4_2 = Conv(filters = 96,
                         kernel_size = (3,3),
                         strides = (2,2),
                         padding = 'valid'
                        )
        
        self.C5_1 = Conv(filters = 64,
                         kernel_size = (1,1)
                        )
        self.C5_2 = Conv(filters = 64,
                         kernel_size = (1,1),
                        )
        self.C6_1 = Conv(filters = 96,
                         kernel_size = (3,3),
                         padding = 'valid'
                        )
        self.C6_2 = Conv(filters = 64,
                         kernel_size = (7,1)
                        )
        self.C7_2 = Conv(filters = 64,
                         kernel_size = (1,7),
                        )
        self.C8_2 = Conv(filters = 96,
                         kernel_size = (3,3),
                         padding = 'valid'
                        )
        
        self.C9_1 = Conv(filters = 192,
                         kernel_size = (3,3),
                         strides = (2,2),
                         padding = 'valid'
                        )
        self.MP9_2 = tf.keras.layers.MaxPool2D(pool_size = (3,3), ###
                                               strides = (2,2),
                                               padding = 'valid'
                                              )
        
    def call(self, X):
        y = self.C1(X)
        y = self.C2(y)
        y = self.C3(y)
        y1 = self.MP4_1(y)
        y2 = self.C4_2(y)
        y = tf.concat([y1,y2], axis = -1)
        
        y1 = self.C5_1(y)
        y1 = self.C6_1(y1)
        y2 = self.C5_2(y)
        y2 = self.C6_2(y2)
        y2 = self.C7_2(y2)
        y2 = self.C8_2(y2)
        y = tf.concat([y1, y2], axis = -1)
        
        y1 = self.C9_1(y)
        y2 = self.MP9_2(y)
        y = tf.concat([y1,y2], axis = -1)
        return y
    
    
class InceptionA(tf.keras.layers.Layer):
    def __init__(self):
        super(InceptionA, self).__init__()
        
        #path1
        self.P1 = tf.keras.Sequential([
            tf.keras.layers.AveragePooling2D(pool_size = (3,3),
                                             strides = (1,1),
                                             padding = 'same'
                                            ),
            Conv(filters = 96,
                 kernel_size = (1,1)
                )
        ])
        self.P2 = Conv(filters = 96,
                       kernel_size = (1,1)
                      )
        self.P3 = tf.keras.Sequential([
            Conv(filters = 64,
                 kernel_size = (1,1)
                ),
            Conv(filters = 96,
                 kernel_size = (3,3)
                )
        ])
        self.P4 = tf.keras.Sequential([
            Conv(filters = 64,
                 kernel_size = (1,1)
                ),
            Conv(filters = 96,
                 kernel_size = (3,3)
                ),
            Conv(filters = 96,
                 kernel_size = (3,3)
                )
        ])
    
    def call(self, X):
        y1 = self.P1(X)
        y2 = self.P2(X)
        y3 = self.P3(X)
        y4 = self.P4(X)
        y = tf.concat([y1,y2,y3,y4], axis = -1)
        return y


class InceptionB(tf.keras.layers.Layer):
    def __init__(self):
        super(InceptionB, self).__init__()
        
        #Path1
        self.P1 = tf.keras.Sequential([
            tf.keras.layers.AveragePooling2D(pool_size = (3,3),
                                             strides = (1,1),
                                             padding = 'same'
                                            ),
            Conv(filters = 128,
                 kernel_size = (1,1),
                )
        ])
        #Path2
        self.P2 = Conv(filters = 384,
                       kernel_size = (1,1)
                      )
        #Path3
        self.P3 = tf.keras.Sequential([
            Conv(filters = 192,
                 kernel_size = (1,1)
                ),
            Conv(filters = 224,
                 kernel_size = (7,1)
                ),
            Conv(filters = 256,
                 kernel_size =  (1,7)
                )
        ])
        #Path4
        self.P4 = tf.keras.Sequential([
            Conv(filters = 192,
                 kernel_size = (1,1)
                ),
            Conv(filters = 192,
                 kernel_size = (1,7)
                ),
            Conv(filters = 224,
                 kernel_size = (7,1)
                ),
            Conv(filters = 224,
                 kernel_size = (1,7)
                ),
            Conv(filters = 256,
                 kernel_size = (7,1)
                )
        ])
        
    def call(self, X):
        y1 = self.P1(X)
        y2 = self.P2(X)
        y3 = self.P3(X)
        y4 = self.P4(X)
        y = tf.concat([y1,y2,y3,y4], axis = -1)
        return y


class InceptionC(tf.keras.layers.Layer):
    def __init__(self):
        super(InceptionC, self).__init__()
        
        #Path1
        self.P1 = tf.keras.Sequential([
            tf.keras.layers.AveragePooling2D(pool_size = (3,3),
                                             strides = (1,1),
                                             padding = 'same'
                                            ),
            Conv(filters = 256,
                 kernel_size = (1,1)
                )
        ])
        #Path2
        self.P2 = Conv(filters = 256,
                       kernel_size = (1,1)
                      )
        #Path3
        self.P3 = Conv(filters = 384,
                         kernel_size = (1,1)
                        )
        self.P3_1 = Conv(filters = 256,
                         kernel_size = (1,3)
                        )
        self.P3_2 = Conv(filters = 256,
                         kernel_size = (3,1)
                        )
        #Path4
        self.P4 = tf.keras.Sequential([
            Conv(filters = 384,
                 kernel_size = (1,1)
                ),
            Conv(filters = 448,
                 kernel_size = (1,3)
                ),
            Conv(filters = 512,
                 kernel_size = (3,1)
                )
        ])
        self.P4_1 = Conv(filters = 256,
                         kernel_size = (3,1)
                        )
        self.P4_2 = Conv(filters = 256,
                         kernel_size = (1,3)
                        )
        
    def call(self, X):
        y1 = self.P1(X)
        y2 = self.P2(X)
        y3 = self.P3(X)
        y3_1 = self.P3_1(y3)
        y3_2 = self.P3_2(y3)
        y4 = self.P4(X)
        y4_1 = self.P4_1(y4)
        y4_2 = self.P4_2(y4)
        y = tf.concat([y1,y2,y3_1,y3_2,y4_1,y4_2], axis = -1)
        return y


class ReductionA(tf.keras.layers.Layer):
    def __init__(self):
        super(ReductionA, self).__init__()
        
        #Path1
        self.P1 = tf.keras.layers.MaxPool2D(pool_size = (3,3),
                                            strides = (2,2),
                                           )
        #Path2
        self.P2 = Conv(filters = 384,
                       kernel_size = (3,3),
                       strides = (2,2),
                       padding = 'valid'
                      )
        #Path3
        self.P3 = tf.keras.Sequential([
            Conv(filters = 192,
                 kernel_size = (1,1)
                ),
            Conv(filters = 224,
                 kernel_size = (3,3)
                ),
            Conv(filters = 256,
                 kernel_size = (3,3),
                 strides = (2,2),
                 padding = 'valid'
                )
        ])
        
    def call(self, X):
        y1 = self.P1(X)
        y2 = self.P2(X)
        y3 = self.P3(X)
        y = tf.concat([y1,y2,y3], axis = -1)
        return y
    

class ReductionB(tf.keras.layers.Layer):
    def __init__(self):
        super(ReductionB, self).__init__()
        
        #Path1
        self.P1 = tf.keras.layers.MaxPool2D(pool_size = (3,3),
                                            strides = (2,2)
                                           )
        #Path2
        self.P2 = tf.keras.Sequential([
            Conv(filters = 192,
                 kernel_size = (1,1)
                ),
            Conv(filters = 192,
                 kernel_size = (3,3),
                 strides = (2,2),
                 padding = 'valid'
                )
        ])
        #Path3
        self.P3 = tf.keras.Sequential([
            Conv(filters = 256,
                 kernel_size = (1,1)
                ),
            Conv(filters = 256,
                 kernel_size = (1,7)
                ),
            Conv(filters = 320,
                 kernel_size = (7,1)
                ),
            Conv(filters = 320,
                 kernel_size = (3,3),
                 strides = (2,2),
                 padding = 'valid'
                )
        ])
        
    def call(self, X):
        y1 = self.P1(X)
        y2 = self.P2(X)
        y3 = self.P3(X)
        y = tf.concat([y1,y2,y3], axis = -1)
        return y

In [4]:
class InceptionV4(tf.keras.models.Model):
    def __init__(self, n_labels:int, last_activation:str = 'softmax'):
        super(InceptionV4, self).__init__()
        self.n_labels = n_labels
        self.last_activation = last_activation
        
        self.Stem = Stem()
        self.InceptionABlocks = tf.keras.Sequential([
            InceptionA() for x in range(4)
        ])
        self.ReductionA = ReductionA()
        self.InceptionBBlocks = tf.keras.Sequential([
            InceptionB() for x in range(7)
        ])
        self.ReductionB = ReductionB()
        self.InceptionCBlocks = tf.keras.Sequential([
            InceptionC() for x in range(3)
        ])
        self.classifier = tf.keras.Sequential([
            tf.keras.layers.AveragePooling2D(pool_size = (8,8),
                                             strides = (1,1)
                                            ),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dropout(.2),
            tf.keras.layers.Dense(self.n_labels, activation = self.last_activation)
        ])
        
    def call(self, X):
        y = self.Stem(X)
        y = self.InceptionABlocks(y)
        y = self.ReductionA(y)
        y = self.InceptionBBlocks(y)
        y = self.ReductionB(y)
        y = self.InceptionCBlocks(y)
        y = self.classifier(y)
        return y

In [5]:
inceptionv4 = InceptionV4(1000)

In [6]:
inceptionv4.build([16,299,299,3])

In [7]:
inceptionv4.summary()

Model: "inception_v4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
stem (Stem)                  multiple                  608320    
_________________________________________________________________
sequential_12 (Sequential)   (16, 35, 35, 384)         1277824   
_________________________________________________________________
reduction_a (ReductionA)     multiple                  2309280   
_________________________________________________________________
sequential_35 (Sequential)   (16, 17, 17, 1024)        20601504  
_________________________________________________________________
reduction_b (ReductionB)     multiple                  2752000   
_________________________________________________________________
sequential_44 (Sequential)   (16, 8, 8, 1536)          13688640  
_________________________________________________________________
sequential_45 (Sequential)   (16, 1000)               